In [ ]:
import os 
import os.path as osp
import numpy as np 
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
import random
import open3d as o3d
import open3d.core as o3c
import json
import subprocess
import pickle
import torch
from tqdm import tqdm
import torch.multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor
#the other imports from the local stuff
import sys

ws_dir = '/local/home/ekoller/BT'
print(ws_dir)
sys.path.append(ws_dir)
from utils import scan3r,visualisation

In [38]:
#reading in the necessary data
data_dir ='/local/home/ekoller/R3Scan'
scenes_dir = '/local/home/ekoller/R3Scan/scenes'
scan_id= "10b1795b-3938-2467-88fe-b10cad8913f8" #is reference scan  since it is a reference scan everything shouls be correctly hit - it isn't tho
frame_number = "000086"
img_width = 960
img_height = 540


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print(torch.cuda.is_available() )

#size of the content:for every frame we get the following: 9x16 size of array which gives every patch a object id
with open('/local/home/ekoller/R3Scan/files/patch_anno/patch_anno_16_9/0cac75ab-8d6f-2d13-8fea-b1eb7e9bf6e7.pkl', 'rb') as f:
    data = pickle.load(f)
    

# Inspect the contents
print(data)


#try out how the data is giveeen from the pixelwise annotation
# gt_2D_obj_anno_imgs = scan3r.load_gt_2D_anno(data_dir, scan_id, 1)
# desired_array = gt_2D_obj_anno_imgs['000091']

# print(desired_array.shape)



# print("now the pixel wise level", gt_2D_obj_anno_imgs)

{'000000': array([[  7,   7,   7,   7,   7, 100, 100,   8, 101, 101, 101,   9,   9,
          9,   1,   1],
       [  7,   7,   7,   7,   7, 100, 100,   8, 101, 101, 101,   9,   9,
          9,   1,   1],
       [  7,   7,   7,   7,   7, 100, 100,   8, 101, 101, 101,   9, 101,
          9,   1,   1],
       [  7,   7,   7,   7,   7,   8,   8,   8, 101, 101, 101, 101,   9,
          9,   1,   1],
       [  7,   7,  38,  38,  38,   8,   8, 101,   9,   9,   9,   9,   9,
          9,   1,   1],
       [ 15,  10,  10,  10,  10,   8,   8,   9,   9,   9,   9,   9,   9,
          1,   1,   1],
       [ 10,  10,  10,  10,  10,  10,   8,   9,   9,   9,   9,   9,   9,
          1,   1,   1],
       [ 10,  10,  10,  10,  10,  10,   9,   9,   9,   9,   9,   9,   9,
          1,   1,   1],
       [ 10,  10,  10,   9,   9,   9,   9,   9,   9,   9,   9,   9,   1,
          1,   1,   1]], dtype=uint8), '000001': array([[  7,   7,   7, 100, 100, 100, 101, 101, 101,   9,   9, 101,   9,
          1,   1, 

In [ ]:
import pickle

#returns the boundingboxes of a scan as a numpy array
def get_bounding_boxes(data_dir,scan_id):
    pklfile = osp.join(data_dir, 'files', 'orig', 'data', '{}.pkl'.format(scan_id) )
    with open(pklfile, "rb") as f:
            # Load the data
            data = pickle.load(f)

    bounding_boxes = data['bounding_boxes']
    bounding_boxes = np.array(bounding_boxes)

   # print("bounding_boxes", bounding_boxes.shape)

    return bounding_boxes




#get_bounding_boxes(data_dir,scan_id)

In [ ]:
# transform from rescan coordinates(assumption) to camera coordinates
def get_box_coord_in_camera_coord(bounding_boxes_input, world_to_camera_mat):
    
    bounding_boxes = bounding_boxes_input.copy()
    # frame_rot = world_to_camera_mat[:3, :3]
    # frame_trans = world_to_camera_mat[:3, 3] * 1000.0
    # cam_center = -np.matmul(np.transpose(frame_rot), frame_trans)
    #cam_centers.append(cam_center / 1000.0)

    #iterate through every box and adjust it
    for i,box in enumerate(bounding_boxes):
        box = scan3r.coord_to_ref(world_to_camera_mat, box,3)

    
    #to do from a rescan to another rescan
            
        

    return bounding_boxes

In [ ]:
# transform from reference to rescan coordinates! prividing path to r3scan
#all boxes are stored in the reference coordinate system
def get_box_coord_in_rescan_coord(data_dir,bounding_boxes, scan_id_from,scan_id_to):
    #same coordinate system
    ref_id = scan3r.get_reference_id(data_dir,scan_id_from)
    #if we want the coords in the reference coordinate system return the boxes (based on pkl file)
    if scan_id_to==ref_id:
        return bounding_boxes
    
    #so as to not modify in polace
    res_boxes = bounding_boxes.copy()

    #transform the centers of rescan to ref coord
    path = osp.join(data_dir,"files", "3RScan.json")
    map_id_to_trans = scan3r.read_transform_mat(path)
    transform_to_to_from = map_id_to_trans[scan_id_to]
    transform_to_to_from= transform_to_to_from.reshape(4,4)

    #invert to get transfromtation form reference to rescan
    transform_from_to_to = np.linalg.inv(transform_to_to_from)

    #iterate through every box and adjust it
    for i,box in enumerate(res_boxes):
        res_boxes[i]= scan3r.coord_to_ref(transform_from_to_to, box,3)

    return res_boxes

#bounding_boxes = get_box_coord_in_rescan_coord(data_dir,get_bounding_boxes(data_dir,"0cac7664-8d6f-2d13-8e1d-b97c9ab20b12"),"0cac7664-8d6f-2d13-8e1d-b97c9ab20b12","dbeb4d1f-faf9-2324-98d1-605c3c0c0658")
#print(bounding_boxes)

In [ ]:
#for a given scan returns the reference, if the scan is reference returns the reference
def get_reference_id(data_dir,scan_id):
    #refscan of a reference is the reference
    if scan3r.is_reference(data_dir, scan_id):
        return scan_id
    
    #open the 3Rjson file
    dir_path = osp.join(data_dir,"files","3RScan.json")
    with open(dir_path, 'r') as file:
        data = json.load(file)

    for item in data:
        if any(scan_id in scan["reference"] for scan in item["scans"]):
            return item["reference"]

    return None


print("get the reference of", get_reference_id(data_dir,scan_id))

    

In [42]:
#returns a ray per pixel as a tensor shot through the pixel given a camera position
def ray_per_pixel(scenes_dir,scan_id, frame_id):
    #get intrinsics:the infofrom the camera
    camera_info = scan3r.load_intrinsics(scenes_dir,scan_id)
    #3x3 matrix
    cam_to_img = camera_info['intrinsic_mat']
    img_width = int(camera_info['width'])
    img_height = int(camera_info['height'])
    print("img height", img_height, "img_width", img_width)

    #get camera to world transformation matrix 4x4 matrix
    camera_to_world_mat = scan3r.load_pose(scenes_dir,scan_id, frame_id)

    #camera pose in the camera coordinate system -> get the last translation column
    camera_pose = camera_to_world_mat[:3, 3]
    print("camera coord", camera_pose)

    #invert the matrix to get extrinsic matrix
    world_to_camera_mat = np.linalg.inv(camera_to_world_mat)
    print("world to camera mat /extrinsic", world_to_camera_mat)

    #create the rays resulting outputshape is  {height_px, width_px, 6} first 3 are the ray origin, later 3 are the direction
    rays = o3d.t.geometry.RaycastingScene.create_rays_pinhole(
            intrinsic_matrix = cam_to_img.astype(np.float64),
            extrinsic_matrix = world_to_camera_mat.astype(np.float64),
            width_px = img_width, height_px = img_height
        )
    
    #print( "rays shape", rays.shape)

    
    #visualization part
    #camera visualization
    camera_mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
    camera_mesh.transform(world_to_camera_mat)

    """
    #ray visualization
    lines = []
    for i in range(0,rays.shape[0],10):  # Iterate over height (rows)
        for j in range(0,rays.shape[1],10):  # Iterate over width (columns)
            origin = rays[i, j, :3].cpu().numpy() # Extract origin of the ray
            direction = rays[i, j, 3:].cpu().numpy()  # Extract direction of the ray
            endpoint = origin + direction * 10  # Scale direction for visualization
            lines.append([origin, endpoint])
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np.vstack(lines)),
        lines=o3d.utility.Vector2iVector(np.arange(len(lines)).reshape(-1, 2)),
    )
   
    o3d.visualization.draw_geometries([camera_mesh, line_set])
    """
    


    return rays
   



#ray_per_pixel(scenes_dir, "02b33e01-be2b-2d54-93fb-4145a709cec5", "000002")


In [ ]:
#return the faces of the bounding boxes
def get_bounding_boxes_faces(bounding_boxes):
    bboxes_faces = []

    #go through every bouding box
    for bbox in bounding_boxes:
        #create all 6 faces and save the coordinates
        faces = [
        [bbox[0], bbox[1], bbox[2], bbox[3]],  # bottom clockwise direction
        [bbox[4], bbox[5], bbox[6], bbox[7]],  # top clockwise direction
        [bbox[0], bbox[1], bbox[5], bbox[4]],  # front clockwise direction
        [bbox[3], bbox[2], bbox[6], bbox[7]],  # back clockwise direction
        [bbox[3], bbox[0], bbox[4], bbox[7]],  # right clockwise diretion
        [bbox[1], bbox[2], bbox[6], bbox[5]],  # left clockwise direction
        ]

        bboxes_faces.append(faces)

    bboxes_faces = np.array(bboxes_faces)
    print("bounding boxes faces shape ",bboxes_faces.shape )

    return  bboxes_faces
        


In [ ]:
#return the closest bounding box which gets intersected by a ray, if no boundingbox hit, return -1
#however the data is put in, it gets returned that way (cpu or gpu) except intersection point is torch tensor
def get_ray_intersection(object_ids, bounding_boxes_faces, ray):
    #go through the boxes
    ray_origin = ray[:3]
    ray_dir = ray[-3:]
    closest_face = (-1, float("inf"))
    intersection_point = np.full(3, float("inf"))
    intersection_point_tensor = torch.tensor(intersection_point,dtype=torch.float32)  # Convert to PyTorch tensor
    intersection_point_tensor = intersection_point_tensor.to(device)
    is_og_intersection = True
   
    
    

    for obj_id,box in zip(object_ids,bounding_boxes_faces):
        #gro through all faces of the box
        for face in box:
    
            #normal vector of face (vectors from face[0] outgoing)
            edge1 = face[1] - face[0]
            edge2 = face[2] - face[0]
            normal = torch.linalg.cross(edge1,edge2)
            #normalization of normal vector
            normal = normal / torch.norm(normal)
            
            #check if the ray is parallel to the face (with some error)
            angle = torch.dot(ray_dir,normal)
            if abs(angle) < 1e-6:
                continue

            #plane equaion ax + by + cz = d, distance to the plane
            plane_eq = torch.dot(normal, face[0])

            #point of intersection: intersection = ray_origion + t* ray direction
            #np.dot(ray_origin,normal): dignes distance from the origin to the plane along the direction of the normal

            t = (plane_eq - torch.dot(ray_origin,normal)) /angle
            if t <0:
                continue
            intersection = ray_origin + (t * ray_dir)

            #calculate the distance
            dist = torch.norm(intersection - ray_origin)

            #compute the edges needed for the check
            e1 = face[1] - face[0]
            e2 = face[2] - face[1]
            e3 = face[3] - face[2]
            e4 = face[0] - face[3]

            # get the vectors from the intersectionpoint to each of the vertecies
            to_0 = face[0] - intersection
            to_1 = face[1] - intersection
            to_2 = face[2] - intersection
            to_3 = face[3] - intersection

            #get the direction of the normal for the to_x and edges
            n1 = torch.linalg.cross(e1, to_0)
            n2 = torch.linalg.cross(e2, to_1)
            n3 = torch.linalg.cross(e3, to_2)
            n4 = torch.linalg.cross(e4, to_3)

            #check that all point into the same direction
            if (torch.dot(n1, n2) >= 0) and (torch.dot(n2, n3) >= 0) and (torch.dot(n3, n4) >= 0) and (torch.dot(n4, n1) >= 0):
                #point is within the face
                #check if it is closer than the currently saved one
                curr_id, curr_closest_dist = closest_face
                if (curr_closest_dist > dist):
                    closest_face = (obj_id,dist)
                    intersection_point_tensor = intersection
                    is_og_intersection = False
                    #print("current closest face" ,closest_face, "current intersection point", intersection_point_tensor)
    
    intersection_point = intersection_point_tensor.cpu().numpy()

   # print("final closest face ", closest_face, "final intersection point ", intersection_point)
    obj_id, dist = closest_face
    return obj_id, intersection_point #tensors

In [ ]:
#experiment to find the bug lol try it out with the rescan boundingboxes to check the bounding box



import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor
#compute the intersection withthe boundingboxes, retruns numpy 
def pixel_wise_bounding_box_intersection(data_dir,scenes_dir,scan_id, frame_number):
    reference = get_reference_id(data_dir,scan_id)

    assert(reference != None)

    #shape (#boxes,8,3), numpy array
    bounding_boxes = get_bounding_boxes(data_dir, reference)
    print("shape of bounding boxes", bounding_boxes.shape)

    #transform the bounding boxes of the reference into the coordinate system of the rescan:
    bounding_boxes_rescan_coord = get_box_coord_in_rescan_coord(data_dir,bounding_boxes, reference,scan_id)

    #transform the bounding boxes from the rescan coordinates into the coordinate system of the camera
    camera_to_world_mat = scan3r.load_pose(scenes_dir,scan_id, frame_number)
    world_to_camera_mat = np.linalg.inv(camera_to_world_mat)
    bounding_boxes_camera_coord =get_box_coord_in_camera_coord(bounding_boxes_rescan_coord, world_to_camera_mat)
    


    #get the faces of the ref boundingboxes in the rescan coordinates
    #shape (#boxes,6,4,3), numpy array
    bounding_boxes_faces = get_bounding_boxes_faces(bounding_boxes_camera_coord)
    print("shape of bounding boxes faces", bounding_boxes_faces.shape)



    #shape (height, width, 6), tensor
    rays = ray_per_pixel(scenes_dir,scan_id, frame_number)
 
   
    #get the object id of the scene
    object_ids = []
    path_to_obj_id = osp.join(scenes_dir, scan_id, 'semseg.v2.json')

    

    with open(path_to_obj_id, 'r') as file:
        data = json.load(file)

    #get the id for the objects visible in the reference scan
    for object in data['segGroups']:
        id = object["objectId"]
        object_ids.append(id)


    print("semseg object id cound ", len(object_ids))
    print("bounding boxes cound ", bounding_boxes.shape)

    #put the rays,bounding box faces and object ids into gpu for this computation
    print("----------start converting from cpu to gpu-----------")
    
    #rays_numpy = np.asarray(rays,dtype=np.float32)  # Convert to NumPy array
    rays_numpy = rays.numpy()
    rays_tensor = torch.tensor(rays_numpy,dtype=torch.float32)  # Convert to PyTorch tensor
    rays_tensor = rays_tensor.to(device)
    print("----------rays done-----------")
    
    #bounding_boxes_faces_array = np.array(bounding_boxes_faces,dtype=np.float32)
    bounding_boxes_faces_array = np.array(bounding_boxes_faces,dtype=np.float32)

    bounding_boxes_faces_tensor = torch.tensor(bounding_boxes_faces_array, dtype=torch.float32)
    bounding_boxes_faces_tensor = bounding_boxes_faces_tensor.to(device)

    print("----------boundin boxes faces done-----------")

    object_ids_array = np.array(object_ids, dtype= np.int32)
    object_ids_tensor = torch.tensor(object_ids_array, dtype= torch.int32)
    object_ids_tensor = object_ids_tensor.to(device)
    print("----------object ids done-----------")


    print("----------finished converting from cpu to gpu-----------")


    #if the scene id is not reference the numbers will differ -> to use the pointcloud of the reference we use the semseg since it has hall the is
    #print("bounding box object numers ", len(bounding_boxes), "seggroup objec length", len(object_ids))



    bounding_box_per_pixel = np.full((img_height, img_width), -1, dtype= np.int32)

    print("shape of bounding box per pixel",bounding_box_per_pixel.shape)

    intersection_points = np.zeros((img_height,img_width,3), dtype= np.float32)

    
    #use a thread pool do make things at least a bit more efficient
    #pool = mp.pool.ThreadPool()


    # define task
    def task (i, j):
        ray_tensor = rays_tensor[i][j]
        bounding_box_per_pixel_res,intersection_points_res= get_ray_intersection(object_ids_tensor, bounding_boxes_faces_tensor, ray_tensor)
        #print("resulting boundinboxperpixel id ", bounding_box_per_pixel_res_tensor, " resulting intersetictionpoint ", intersection_points_res)
        bounding_box_per_pixel[i][j] = bounding_box_per_pixel_res
        #print(" put back to cpu ids ", bounding_box_per_pixel[i][j])
        intersection_points[i][j] = intersection_points_res
        #print("bounding box per pixel (", i, "," , j, ")",bounding_box_per_pixel[i][j], " intersection point ij ", intersection_points[i][j])
        progress_bar.update(1)

    
    total_tasks = (img_height//30)* (img_width//30)
    # Create a progress bar
    progress_bar = tqdm(total=total_tasks, desc="Processing", unit="pixels")

    args = [(i, j) for i in range(0, img_height, 30) for j in range(0, img_width, 30)]

    with ThreadPoolExecutor() as executor:
        executor.map(task, *zip(*args))


    # print("print within the boundingboxperpixel boundingbox ", bounding_box_per_pixel)
    # for i in range(0, img_height, 30):
    #     for j in range(0, img_width, 30):
    #        print("element (" , i ,",", j,") is", bounding_box_per_pixel[i][j])
    


    #print("bounding_boy_per_pixel", bounding_box_per_pixel)
    return bounding_box_per_pixel,intersection_points

In [ ]:
#this code returns the colour segmentation of the gt projection on pixelwise level
def load_gt_2D_anno_img(data_root_dir, scan_id, skip=None):
    import cv2
    anno_imgs = {}
    frame_idxs = scan3r.load_frame_idxs(osp.join(data_root_dir, "scenes"), scan_id, skip)
    anno_folder = osp.join(data_root_dir, "files", 'gt_projection/color', scan_id)
    for frame_idx in frame_idxs:
        anno_img_file = osp.join(anno_folder, "frame-{}.jpg".format(frame_idx))
        anno_img = cv2.imread(anno_img_file, cv2.IMREAD_UNCHANGED)
        anno_imgs[frame_idx] = anno_img
    return anno_imgs

In [16]:
#this code segment shows us the following: sanity check if the ray shooting works correctly: for a given scan_id which is a reference scan, look which boxes are correctly hit
result_id,result_intersection_points =pixel_wise_bounding_box_intersection(data_dir,scenes_dir,scan_id, frame_number)
print("resulting ids ",result_id)
print("resulting intersection points ", result_intersection_points)

shape of bounding boxes (34, 8, 3)
bounding boxes faces shape  (34, 6, 4, 3)
shape of bounding boxes faces (34, 6, 4, 3)
img height 540 img_width 960
camera coord [-0.635085  0.445091  0.143526]
world to camera mat /extrinsic [[-0.46795042  0.36049579 -0.80688681 -0.34183249]
 [ 0.55951842  0.82758102  0.04525072 -0.01950176]
 [ 0.6840771  -0.43029264 -0.58897072  0.7104991 ]
 [ 0.          0.          0.          1.        ]]
semseg object id cound  34
bounding boxes cound  (34, 8, 3)
----------start converting from cpu to gpu-----------
----------rays done-----------
----------boundin boxes faces done-----------
----------object ids done-----------
----------finished converting from cpu to gpu-----------
shape of bounding box per pixel (540, 960)


Processing: 100%|██████████| 576/576 [00:28<00:00, 19.93pixels/s]

resulting ids  [[ 7 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 ...
 [-1 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]]
resulting intersection points  [[[-0.35522425  0.107135    0.111614  ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  ...
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  ...
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  ...
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 ...

 [[ 0.          0.          0.        ]
  [ 0.  

In [ ]:
#based on result id above
#this code segment  calculates the boundingboxintersections and prints the resulting images 
gt_2D_obj_anno_id = scan3r.load_gt_2D_anno(data_dir, scan_id, 1)
gt_array = gt_2D_obj_anno_id[frame_number]

#get also the colour images
gt_2D_obj_anno_img = load_gt_2D_anno_img(data_dir, scan_id, 1)
desired_img = gt_2D_obj_anno_img[frame_number]

#load the image into cv2 and also create the current output image which has all non correctly determined points black the rest colourful
gt_img = np.array(desired_img)
gt_img_resized = gt_img[::30, ::30] #resize the image since we look only at every 30th element
print("ScanId ", scan_id,  "framenumber ", frame_number)

#to make the displayed window bigger
gt_image_resized_large = cv2.resize(gt_img_resized, (960, 540))
cv2.imshow("Input Image",  gt_image_resized_large)

res_img = np.zeros_like(gt_img)
res_img_resized = res_img[::30, ::30] #resize the image since we look only at every 30th element

for i in range(0,img_height,30):
        for j in range(0,img_width,30):
                if gt_array[i][j] == result_id[i][j]:
                        #set the pixel
                        res_img_resized[i//30][j//30] = gt_img_resized[i//30][j//30]




# Display the output image
res_image_resized_large = cv2.resize(res_img_resized, (960, 540))
cv2.imshow('Output Image', res_image_resized_large)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
#print true and false of whatever happened / got computed
#this code segment shows us the following: sanity check if the ray shooting works correctly: for a given scan_id which is a reference scan, look which boxes are correctly hit
gt_2D_obj_anno_id = scan3r.load_gt_2D_anno(data_dir, scan_id, 1)
desired_array = gt_2D_obj_anno_id[frame_number]

#get also the colour images
gt_2D_obj_anno_img = load_gt_2D_anno_img(data_dir, scan_id, 1)
desired_img = gt_2D_obj_anno_img[frame_number]

#load the image into cv2 and also create the current output image which has all non correctly determined points black the rest colourful
gt_img = np.array(desired_img)
res_img = np.zeros_like(gt_img)

result_id,result_intersection_points =pixel_wise_bounding_box_intersection(data_dir,scenes_dir,scan_id, frame_number)
print("resulting ids ",result_id)
print("resulting intersection points ", result_intersection_points)

rsult_type = result_id.dtype
print("type of result id ", rsult_type)



for i in range(0,img_height,30):
        for j in range(0,img_width,30):
                if desired_array[i][j] != result_id[i][j]:
                        print( False, "desired was ", desired_array[i][j], "we got", result_id[i][j]) 
                else:
                        print( True, "desired was ", desired_array[i][j], "we got", result_id[i][j])
                        #set the pixel
                        res_img[i][j] = gt_img[i][j]
                        print("res img_colour is ", res_img[i][j])


print("resulting image after foor loop " ,res_img)
# #resize sinze we dont compute all the rays 
# new_size = (img_height/30, img_width/30)


# gt_img = cv2.resize(gt_img,(gt_img.shape[1] // 30, gt_img.shape[0] // 30))
# res_img = cv2.resize(res_img,(res_img.shape[1] // 30, res_img.shape[0] // 30))

# # Display the input and output images
# cv2.imshow('Input Image', gt_img)
# cv2.imshow('Output Image', res_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

shape of bounding boxes (34, 8, 3)
bounding boxes faces shape  (34, 6, 4, 3)
shape of bounding boxes faces (34, 6, 4, 3)
img height 540 img_width 960
camera coord [-0.635085  0.445091  0.143526]
world to camera mat /extrinsic [[-0.46795042  0.36049579 -0.80688681 -0.34183249]
 [ 0.55951842  0.82758102  0.04525072 -0.01950176]
 [ 0.6840771  -0.43029264 -0.58897072  0.7104991 ]
 [ 0.          0.          0.          1.        ]]
semseg object id cound  34
bounding boxes cound  (34, 8, 3)
----------start converting from cpu to gpu-----------
----------rays done-----------
----------boundin boxes faces done-----------
----------object ids done-----------
----------finished converting from cpu to gpu-----------
shape of bounding box per pixel (540, 960)


Processing: 100%|██████████| 576/576 [00:29<00:00, 19.40pixels/s]

resulting ids  [[ 7 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 ...
 [-1 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]
 [-1 -1 -1 ... -1 -1 -1]]
resulting intersection points  [[[-0.35522425  0.107135    0.111614  ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  ...
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  ...
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 [[ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  ...
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]
  [ 0.          0.          0.        ]]

 ...

 [[ 0.          0.          0.        ]
  [ 0.  

In [18]:
#transformation from rescan coordinates to camera coordinates plus  ray visualization
def visualize_rescan_to_camera(data_dir,reference_id,scan_id, frame_id):
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    #visualization of the camera rays
    camera_to_world_mat = scan3r.load_pose(scenes_dir,scan_id, frame_id)
    world_to_camera_mat = np.linalg.inv(camera_to_world_mat)

    #camera pose in the camera coordinate system -> get the last translation column
    camera_pose = camera_to_world_mat[:3, 3]
    print("camera coord", camera_pose)
    
    camera_position = np.array(camera_pose)
    print("camera_position ", camera_position)
    camera_point = o3d.geometry.PointCloud()
    camera_point.points = o3d.utility.Vector3dVector([camera_position])

    coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)
    vis.add_geometry(coordinate_frame)

    vis.add_geometry(camera_point)


    rays = ray_per_pixel(scenes_dir,scan_id, frame_id)
    
    #ray visualization
    lines = []
    for i in range(0,rays.shape[0],60):  # Iterate over height (rows)
        for j in range(0,rays.shape[1],60):  # Iterate over width (columns)
            origin = rays[i, j, :3].cpu().numpy() # Extract origin of the ray
            direction = rays[i, j, 3:].cpu().numpy()  # Extract direction of the ray
            endpoint = origin + direction * 2  # Scale direction for visualization
            lines.append([origin, endpoint])

    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np.vstack(lines)),
        lines=o3d.utility.Vector2iVector(np.arange(len(lines)).reshape(-1, 2)),
    )
    vis.add_geometry(line_set)

    

    intersection_id, intersection_point = pixel_wise_bounding_box_intersection(data_dir,scenes_dir,scan_id, frame_number)

    # # # get the intersetion  points and display them 

    for i in range(0,img_height,60):  # Iterate over height (rows)
        for j in range(0,img_width,60):  # Iterate over width (columns)
           if intersection_id[i][j] != -1:
                 intersect_point = o3d.geometry.PointCloud()
                 intersection = np.array(intersection_point[i][j])
                 intersect_point.points = o3d.utility.Vector3dVector([intersection])
                 vis.add_geometry(intersect_point)


            
    

    #turn the bounding box of the reference scan into the coordinate system of the rescan
    bounding_boxes = get_bounding_boxes(data_dir, reference_id)
    rescan_coord_bounding_boxes = get_box_coord_in_rescan_coord(data_dir,bounding_boxes,reference_id, scan_id)
    camera_cord_bounding_boxes = get_box_coord_in_camera_coord(rescan_coord_bounding_boxes,world_to_camera_mat)

    #get the bounding boxes of the rescan
    bounding_boxes_scan_id = get_bounding_boxes(data_dir, scan_id)
    rescan_coord_bounding_boxes_scan_id = get_box_coord_in_rescan_coord(data_dir,bounding_boxes_scan_id,scan_id, scan_id)
    camera_cord_bounding_boxes_scan_id = get_box_coord_in_camera_coord(rescan_coord_bounding_boxes_scan_id,world_to_camera_mat)


    #define the edges
    edges = [ 
        [0, 1], [1, 2], [2, 3], [3, 0],  # Bottom face
        [4, 5], [5, 6], [6, 7], [7, 4],  # Top face
        [0, 4], [1, 5], [2, 6], [3, 7]   # Connecting edges
        ]


    #print the boundingboxes of reference scan - rescan coord, camera coord black
    for bbox in camera_cord_bounding_boxes:
            #bbox_color = np.array([0,0,0])
            bbox_color = np.random.rand(3)
            line_set = o3d.geometry.LineSet()
            line_set.points.extend(o3d.utility.Vector3dVector(bbox))
            line_set.lines.extend(o3d.utility.Vector2iVector(edges))
            line_set.colors = o3d.utility.Vector3dVector([bbox_color] * len(edges))
            vis.add_geometry(line_set)


  #print the boundin boxes of the rescan in camera coord red
    # for bbox in camera_cord_bounding_boxes_scan_id:
    #         bbox_color = np.array([255,0,0])
    #         line_set = o3d.geometry.LineSet()
    #         line_set.points.extend(o3d.utility.Vector3dVector(bbox))
    #         line_set.lines.extend(o3d.utility.Vector2iVector(edges))
    #         line_set.colors = o3d.utility.Vector3dVector([bbox_color] * len(edges))
    #         vis.add_geometry(line_set)

    #bounding boxes ref scan in ref coordinates
    # for bbox in bounding_boxes:
    #         bbox_color = np.array([0,0,255])
    #         line_set = o3d.geometry.LineSet()
    #         line_set.points.extend(o3d.utility.Vector3dVector(bbox))
    #         line_set.lines.extend(o3d.utility.Vector2iVector(edges))
    #         line_set.colors = o3d.utility.Vector3dVector([bbox_color] * len(edges))
    #         vis.add_geometry(line_set)
    
    #bounding boxes rescan scan in ref coordinates
    # for bbox in bounding_boxes_scan_id:
    #         bbox_color = np.array([255,0,0])
    #         line_set = o3d.geometry.LineSet()
    #         line_set.points.extend(o3d.utility.Vector3dVector(bbox))
    #         line_set.lines.extend(o3d.utility.Vector2iVector(edges))
    #         line_set.colors = o3d.utility.Vector3dVector([bbox_color] * len(edges))
    #         vis.add_geometry(line_set)

        


  

    
    vis.run()
    vis.destroy_window()


reference_id = get_reference_id(data_dir,scan_id)
visualize_rescan_to_camera(data_dir,reference_id, scan_id,frame_number)

camera coord [-0.635085  0.445091  0.143526]
camera_position  [-0.635085  0.445091  0.143526]
img height 540 img_width 960
camera coord [-0.635085  0.445091  0.143526]
world to camera mat /extrinsic [[-0.46795042  0.36049579 -0.80688681 -0.34183249]
 [ 0.55951842  0.82758102  0.04525072 -0.01950176]
 [ 0.6840771  -0.43029264 -0.58897072  0.7104991 ]
 [ 0.          0.          0.          1.        ]]
shape of bounding boxes (34, 8, 3)
bounding boxes faces shape  (34, 6, 4, 3)
shape of bounding boxes faces (34, 6, 4, 3)
img height 540 img_width 960
camera coord [-0.635085  0.445091  0.143526]
world to camera mat /extrinsic [[-0.46795042  0.36049579 -0.80688681 -0.34183249]
 [ 0.55951842  0.82758102  0.04525072 -0.01950176]
 [ 0.6840771  -0.43029264 -0.58897072  0.7104991 ]
 [ 0.          0.          0.          1.        ]]
semseg object id cound  34
bounding boxes cound  (34, 8, 3)
----------start converting from cpu to gpu-----------
----------rays done-----------
----------boundin bo

Processing: 100%|██████████| 576/576 [00:29<00:00, 19.52pixels/s]


In [43]:
#the colours are the same as in the ground truth
#this code segment should visualize the mesh of the given scene in the original scan coordinates for ref it is ref for rescan it is rescan
"""
Mesh preparation and display adding
"""
pathToMesh = osp.join(data_dir,"scenes", scan_id, "labels.instances.annotated.v2.ply")
mesh = o3d.io.read_triangle_mesh(pathToMesh)
# Visualize the mesh

#check if it has colours
if not mesh.has_vertex_colors():
    print("Mesh does not have vertex colors")
    exit()

#make sure the colours are a oke
colors = np.asarray(mesh.vertex_colors)
if np.max(colors) > 1.0:  # Assuming colors are in the range [0, 255]
        colors /= 255.0

#bruh swap the channels bc the colours from gt didn't match like zurias hint -> open: wtf does that mean for the rest
colors = np.asarray(mesh.vertex_colors)
colors = colors[:, [2, 1, 0]]  # Swap red and blue channels
mesh.vertex_colors = o3d.utility.Vector3dVector(colors)


"""
Rays Preparation and display adding
"""
#visualization of the camera rays
camera_to_world_mat = scan3r.load_pose(scenes_dir,scan_id, frame_number)
world_to_camera_mat = np.linalg.inv(camera_to_world_mat)

rays = ray_per_pixel(scenes_dir,scan_id, frame_number)
    
#ray visualization
lines = []
for i in range(0,rays.shape[0],10):  # Iterate over height (rows)
        for j in range(0,rays.shape[1],10):  # Iterate over width (columns)
            origin = rays[i, j, :3].cpu().numpy() # Extract origin of the ray
            direction = rays[i, j, 3:].cpu().numpy()  # Extract direction of the ray
            endpoint = origin + direction * 2  # Scale direction for visualization
            lines.append([origin, endpoint])

line_set_ray = o3d.geometry.LineSet(
points=o3d.utility.Vector3dVector(np.vstack(lines)),
lines=o3d.utility.Vector2iVector(np.arange(len(lines)).reshape(-1, 2)),
    )

 






"""
Bounding Boxes preparation and display adding
"""
#also get the boundingboxes for the scan 
bounding_boxes_scan_id = get_bounding_boxes(data_dir, scan_id)
#turn them into the correct coordinate system
rescan_coord_bounding_boxes_scan_id = get_box_coord_in_rescan_coord(data_dir,bounding_boxes_scan_id,scan_id, scan_id)
camera_coord_bounding_boxes_scan_id = get_box_coord_in_camera_coord(rescan_coord_bounding_boxes_scan_id,world_to_camera_mat)




#define the edges
edges = [ 
        [0, 1], [1, 2], [2, 3], [3, 0],  # Bottom face
        [4, 5], [5, 6], [6, 7], [7, 4],  # Top face
        [0, 4], [1, 5], [2, 6], [3, 7]   # Connecting edges
    ]



#print the boundingboxes of reference scan - rescan coord, camera coord black in random colours
boundingBox = []
for bbox in camera_coord_bounding_boxes_scan_id:
        #bbox_color = np.array([0,0,0])
        bbox_color = np.random.rand(3)
        line_set = o3d.geometry.LineSet()
        line_set.points.extend(o3d.utility.Vector3dVector(bbox))
        line_set.lines.extend(o3d.utility.Vector2iVector(edges))
        line_set.colors = o3d.utility.Vector3dVector([bbox_color] * len(edges))
        boundingBox.append(line_set)


##visualize everything
vis = o3d.visualization.Visualizer()
vis.create_window(width=800, height=600)

vis.add_geometry(mesh)

for bbox in boundingBox:
    vis.add_geometry(bbox)

vis.add_geometry(line_set_ray)

vis.run()
vis.destroy_window()


img height 540 img_width 960
camera coord [-0.960072  0.612676  0.177708]
world to camera mat /extrinsic [[ 0.28004461  0.36791668 -0.88668678  0.20102061]
 [ 0.83174587 -0.55419098  0.03273956  1.13225736]
 [-0.47934844 -0.74666573 -0.46121053  0.07921596]
 [ 0.          0.          0.          1.        ]]


In [ ]:
#sanity check that the transformation to the rescan coordinates does indeed work
#show the boundingboxes of the reference scan in the rescan coordinate system
#given a Reference and a Rescan plotts both pointclouds each in one colour and the corresponding bounding boxes
vis = o3d.visualization.Visualizer()
vis.create_window()




#load the bounding boxes transformed to the reference scan from the original data
reference_id =get_reference_id(data_dir,scan_id)
print("reference id is ", reference_id, " for rescan id ", scan_id)

#load the data 1 for the rescan pointcloud this is still in reference coordinate system
#to do get it in the original way
pklfile = osp.join(data_dir, 'files', 'orig', 'data', '{}.pkl'.format(reference_id) )
with open(pklfile, "rb") as f:
        # Load the data
        data = pickle.load(f)
        pointclouds_saved_ref= data['obj_points'][512]
        bounding_boxes_saved_ref = data['bounding_boxes'] 

print("reference id is ", reference_id, "for the rescan id ", scan_id)

pklfile = osp.join(data_dir, 'files', 'orig', 'data', '{}.pkl'.format(scan_id) )
with open(pklfile, "rb") as f:
        # Load the data
        data = pickle.load(f)
        pointclouds_saved = data['obj_points'][512]
        bounding_boxes_saved_scan_id = data['bounding_boxes'] 

#transform the bounding boxes from the reference coordinate system into the rescan coordinate system
bounding_boxes_ref_in_rescan_coord_ref = get_box_coord_in_rescan_coord(data_dir,bounding_boxes_saved_ref, reference_id,scan_id)
bounding_boxes_ref_in_rescan_coord_scan_id = get_box_coord_in_rescan_coord(data_dir,bounding_boxes_saved_scan_id, reference_id,scan_id)




#transform even further
camera_to_world_mat = scan3r.load_pose(scenes_dir,scan_id, frame_number)
world_to_camera_mat = np.linalg.inv(camera_to_world_mat)
camera_position =  camera_to_world_mat[:3, 3]

#turn all boudningboxes to camera for ref
bounding_boxes_ref_to_rescan_to_cam_ref = get_box_coord_in_camera_coord(bounding_boxes_ref_in_rescan_coord_ref,world_to_camera_mat)
bounding_boxes_ref_to_cam_ref = get_box_coord_in_camera_coord(bounding_boxes_saved_ref, world_to_camera_mat)
#turn all boundingboxes to camera for scabnid
bounding_boxes_ref_to_rescan_to_cam_scan_id = get_box_coord_in_camera_coord(bounding_boxes_ref_in_rescan_coord_scan_id,world_to_camera_mat)
bounding_boxes_ref_to_cam_scan_id = get_box_coord_in_camera_coord(bounding_boxes_saved_scan_id, world_to_camera_mat)



 

#define the edges
edges = [ 
    [0, 1], [1, 2], [2, 3], [3, 0],  # Bottom face
    [4, 5], [5, 6], [6, 7], [7, 4],  # Top face
    [0, 4], [1, 5], [2, 6], [3, 7]   # Connecting edges
    ]


#boundingboxes ref in ref coordinates
# for bbox in bounding_boxes_saved_ref:
#         bbox_color = np.array([50,0,50])
#         line_set = o3d.geometry.LineSet()
#         line_set.points.extend(o3d.utility.Vector3dVector(bbox))
#         line_set.lines.extend(o3d.utility.Vector2iVector(edges))
#         line_set.colors = o3d.utility.Vector3dVector([bbox_color] * len(edges))
#         vis.add_geometry(line_set)

#boundingboxes scan_id in ref coordinates
# for bbox in bounding_boxes_saved_scan_id:
#         bbox_color = np.array([0,0,50])
#         line_set = o3d.geometry.LineSet()
#         line_set.points.extend(o3d.utility.Vector3dVector(bbox))
#         line_set.lines.extend(o3d.utility.Vector2iVector(edges))
#         line_set.colors = o3d.utility.Vector3dVector([bbox_color] * len(edges))
#         vis.add_geometry(line_set)

# #boundingboxes ref in rescan coordinates
# for bbox in bounding_boxes_ref_in_rescan_coord_ref:
#         bbox_color = np.array([50,0,50])
#         line_set = o3d.geometry.LineSet()
#         line_set.points.extend(o3d.utility.Vector3dVector(bbox))
#         line_set.lines.extend(o3d.utility.Vector2iVector(edges))
#         line_set.colors = o3d.utility.Vector3dVector([bbox_color] * len(edges))
#         vis.add_geometry(line_set)


# # boudning boxes scan_id in rescan coordinates
for bbox in bounding_boxes_ref_in_rescan_coord_scan_id:
        bbox_color = np.array([0,255,0])
        line_set = o3d.geometry.LineSet()
        line_set.points.extend(o3d.utility.Vector3dVector(bbox))
        line_set.lines.extend(o3d.utility.Vector2iVector(edges))
        line_set.colors = o3d.utility.Vector3dVector([bbox_color] * len(edges))
        vis.add_geometry(line_set)

# # bounding boxes of ref from ref-rescan-camera coord
# for bbox in bounding_boxes_ref_to_rescan_to_cam_ref: 
#         bbox_color = np.array([50,0,50])
#         line_set = o3d.geometry.LineSet()
#         line_set.points.extend(o3d.utility.Vector3dVector(bbox))
#         line_set.lines.extend(o3d.utility.Vector2iVector(edges))
#         line_set.colors = o3d.utility.Vector3dVector([bbox_color] * len(edges))
#         vis.add_geometry(line_set)


# # bounding boxes of scan_id from ref-rescan-camera coord
for bbox in bounding_boxes_ref_to_rescan_to_cam_scan_id: 
        bbox_color = np.array([0,0,0])
        line_set = o3d.geometry.LineSet()
        line_set.points.extend(o3d.utility.Vector3dVector(bbox))
        line_set.lines.extend(o3d.utility.Vector2iVector(edges))
        line_set.colors = o3d.utility.Vector3dVector([bbox_color] * len(edges))
        vis.add_geometry(line_set)


# # bounding boxes of ref from ref to camera coord
# for bbox in bounding_boxes_ref_to_cam_ref: 
#         bbox_color = np.array([50,0,50])
#         line_set = o3d.geometry.LineSet()
#         line_set.points.extend(o3d.utility.Vector3dVector(bbox))
#         line_set.lines.extend(o3d.utility.Vector2iVector(edges))
#         line_set.colors = o3d.utility.Vector3dVector([bbox_color] * len(edges))
#         vis.add_geometry(line_set)


# # bounding boxes of scan_id from ref to camera coord
# for bbox in bounding_boxes_ref_to_cam_scan_id: 
#         bbox_color = np.array([0,255,0])
#         line_set = o3d.geometry.LineSet()
#         line_set.points.extend(o3d.utility.Vector3dVector(bbox))
#         line_set.lines.extend(o3d.utility.Vector2iVector(edges))
#         line_set.colors = o3d.utility.Vector3dVector([bbox_color] * len(edges))
#         vis.add_geometry(line_set)


# camera_position = np.array(camera_position)
# print("camera_position ", camera_position)
# camera_point = o3d.geometry.PointCloud()
# camera_point.points = o3d.utility.Vector3dVector([camera_position])

# coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2.0)
# vis.add_geometry(coordinate_frame)

# vis.add_geometry(camera_point)


#add the pointcloud from the pkl


# # pointcloud for the reference from pkl
# for points in pointclouds_saved_ref:
#         color = color = np.array([50,0,50])
#         pcd = o3d.geometry.PointCloud()
#         pcd.points = o3d.utility.Vector3dVector(points)
#         pcd.colors = o3d.utility.Vector3dVector(np.tile(color, (len(points), 1))) 
#         vis.add_geometry(pcd)

# pointcloud for the rescan from pkl
# for points in pointclouds_saved:
#         color = np.array([0,0,0])
#         pcd = o3d.geometry.PointCloud()
#         pcd.points = o3d.utility.Vector3dVector(points)
#         pcd.colors = o3d.utility.Vector3dVector(np.tile(color, (len(points), 1))) 
#         vis.add_geometry(pcd)





 
   

   


    # Run the visualizer
vis.run()
vis.destroy_window()